<a href="https://colab.research.google.com/github/campeone-pixel/soccer_database/blob/master/datasets_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import sqlite3
import pandas as pd
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'



!pip install thefuzz
!pip install rapidfuzz
!pip install python-Levenshtein

from rapidfuzz import process, fuzz

# df_games = pd.read_csv("originales/games.csv")
# df_teams = pd.read_csv("originales/teams.csv")
# df_teamstats = pd.read_csv("originales/teamstats.csv")
# df_leagues = pd.read_csv("originales/leagues.csv")
# df_appearances = pd.read_csv("originales/appearances.csv")
# df_players = pd.read_csv("originales/players.csv")
# df_shots = pd.read_csv("originales/shots.csv")


df_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/games.csv")
df_teams = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teams.csv")
df_teamstats = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teamstats.csv")
df_leagues = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/leagues.csv")
df_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/appearances.csv")
df_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/players.csv", encoding="ISO-8859-1")
df_shots = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/shots.csv")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
df_teamstats_away = df_teamstats.loc[df_teamstats["location"] == "a"]
df_teamstats_home = df_teamstats.loc[df_teamstats["location"] == "h"]
df_teamstats_merged = pd.merge(
    df_teamstats_home, df_teamstats_away, on="gameID", suffixes=["_home", "_away"]
)

df_games_merged_with_teamstats = pd.merge(
    df_games, df_teamstats_merged, on="gameID", suffixes=("", "_remove")
)
#df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    #df_games_merged_with_teamstats.iloc[:, 13:34], axis=1
#)
#df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    #["homeProbability", "drawProbability", "awayProbability"], axis=1
#)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    ["teamID_home", "teamID_away"], axis=1
)

df_games_merged_with_leagues = pd.merge(
    df_games_merged_with_teamstats, df_leagues, on="leagueID"
)

df_games_merged_with_team = pd.merge(
    df_games_merged_with_leagues,
    df_teams,
    left_on=["homeTeamID"],
    right_on=["teamID"],
    how="left",
)
df_games_merged_with_team = pd.merge(
    df_games_merged_with_team,
    df_teams,
    left_on=["awayTeamID"],
    right_on=["teamID"],
    how="left",
)

df_f_game = df_games_merged_with_team.rename(
    columns={"name_x": "liga", "name_y": "home_name", "name": "away_name"}
)
df_f_game = df_f_game.drop(
    [
        "understatNotation",
        "season_home",
        "date_home",
        "location_home",
        "goals_away",
        "goals_home",
        "season_away",
        "date_away",
        "location_away",
        "location_away",
        "teamID_x",
        "teamID_y",
    ],
    axis=1,
)


# df_appearances = pd.read_csv("appearances.csv")
# df_players = pd.read_csv("players.csv", encoding="ISO-8859-1")
# df_shots = pd.read_csv("shots.csv")

""" df_apperance_merged_with_players = pd.merge(
    df_appearances, df_players, on="playerID"
) """





' df_apperance_merged_with_players = pd.merge(\n    df_appearances, df_players, on="playerID"\n) '

In [10]:
# df2_appearances = pd.read_csv("players/appearances.csv")

# df2_clubs = pd.read_csv("players/clubs.csv")
# df2_competitions = pd.read_csv("players/competitions.csv")
# df2_games = pd.read_csv("players/games.csv")
# df2_player_val = pd.read_csv("players/player_valuations.csv")
# df2_players = pd.read_csv("players/players.csv")


df2_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/appearances.csv")

df2_clubs = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/clubs.csv")
df2_competitions = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/competitions.csv")
df2_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/games.csv")
df2_player_val= pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/player_valuations.csv")
df2_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/players.csv")


In [11]:

df2_appearances.isna().sum()

appearance_id            0
game_id                  0
player_id                0
player_club_id           0
date                    73
player_pretty_name    2070
competition_id           0
yellow_cards             0
red_cards                0
goals                    0
assists                  0
minutes_played           0
dtype: int64

In [12]:
top_5_ligas = df2_competitions.loc[
    (df2_competitions["pretty_name"] == "Bundesliga")
    | (df2_competitions["pretty_name"] == "Laliga")
    | (df2_competitions["pretty_name"] == "Premier League")
    | (df2_competitions["pretty_name"] == "Serie A")
    | (df2_competitions["pretty_name"] == "Ligue 1")
]


def match_ligue(x, col, score):
    match = process.extractOne(x, col, score_cutoff=score, scorer=fuzz.token_sort_ratio)
    if match:
        return match[2]
    else:
        return 2


# ---------------------------------------------------------------------------------------------------------
# top_5_ligas["pretty_name"]      df_teams.name
top_5_ligas["liga_new_id"] = df_leagues.loc[
    top_5_ligas["pretty_name"]
    .apply(lambda x: match_ligue(x, df_leagues.name, 50))
    .values,
    ["leagueID"],
].values
# ---------------------------------------------------------------------------------------------------------


df2_clubs_top5 = df2_clubs[
    df2_clubs["domestic_competition_id"].isin(top_5_ligas["competition_id"])
]


df2_games_top5 = df2_games[
    df2_games["competition_id"].isin(top_5_ligas["competition_id"])
]
seasons = df_f_game["season"].unique()


df2_games_top5 = df2_games_top5[df2_games_top5["season"].isin(seasons)]



In [13]:
df2_games_top5 = pd.merge(df2_games_top5, top_5_ligas, on="competition_id")


df2_appearances = df2_appearances[
    df2_appearances["game_id"].isin(df2_games_top5["game_id"])
]
#df2_players = df2_players[
    #df2_players["current_club_id"].isin(df2_clubs_top5["club_id"])
#]





#df2_players_val = df2_player_val[
    #df2_player_val["player_id"].isin(df2_players["player_id"])
#]


df2_games_top5["date"] = pd.to_datetime(df2_games_top5["date"], format="%Y/%m/%d")





df_f_game["match_name"] = df_f_game["home_name"] + " vs " + df_f_game["away_name"]

nan_values = pd.DataFrame(np.nan, index=[0], columns=df_f_game.columns)

df_f_game = df_f_game.append(nan_values, ignore_index=True)
df_f_game['date'] = pd.to_datetime(df_f_game['date']).dt.date


In [14]:
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Turin", "club_home_pretty_name"
] = "Torino"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Turin", "club_away_pretty_name"
] = "Torino"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"].str.contains("Mailand"),
    "club_away_pretty_name",
] = "Ac Milan"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"].str.contains("Mailand"),
    "club_home_pretty_name",
] = "Ac Milan"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"].str.contains("Juventus"),
    "club_away_pretty_name",
] = "Juventus"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"].str.contains("Juventus"),
    "club_home_pretty_name",
] = "Juventus"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"].str.contains("Fc Granada"),
    "club_away_pretty_name",
] = "Granada"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"].str.contains("Fc Granada"),
    "club_home_pretty_name",
] = "Granada"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Real Sociedad San Sebastian",
    "club_away_pretty_name",
] = "Real Sociedad"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Real Sociedad San Sebastian",
    "club_home_pretty_name",
] = "Real Sociedad"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Stoke City", "club_away_pretty_name"
] = "Stoke"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Stoke City", "club_home_pretty_name"
] = "Stoke"

df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Swansea City", "club_away_pretty_name"
] = "Swansea"
df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Swansea City", "club_home_pretty_name"
] = "Swansea"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ud Las Palmas", "club_home_pretty_name"
] = "Las Palmas"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ud Las Palmas", "club_away_pretty_name"
] = "Las Palmas"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ssc Neapel", "club_home_pretty_name"
] = "Napoli"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ssc Neapel", "club_away_pretty_name"
] = "Napoli"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Hellas Verona", "club_home_pretty_name"
] = "Verona"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Hellas Verona", "club_away_pretty_name"
] = "Verona"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Us Lecce", "club_home_pretty_name"
] = "Lecce"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Us Lecce", "club_away_pretty_name"
] = "Lecce"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Atalanta Bergamo",
    "club_home_pretty_name",
] = "Atalanta"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Atalanta Bergamo",
    "club_away_pretty_name",
] = "Atalanta"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Brescia Calcio", "club_home_pretty_name"
] = "Brescia"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Brescia Calcio", "club_away_pretty_name"
] = "Brescia"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ac Florenz", "club_home_pretty_name"
] = "Fiorentina"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ac Florenz", "club_away_pretty_name"
] = "Fiorentina"


df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Spal", "club_home_pretty_name"
] = "SPAL 2013"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Spal", "club_away_pretty_name"
] = "SPAL 2013"


df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "As Rom", "club_home_pretty_name"
] = "roma"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "As Rom", "club_away_pretty_name"
] = "roma"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Us Sassuolo", "club_home_pretty_name"
] = "Sassuolo"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Us Sassuolo", "club_away_pretty_name"
] = "Sassuolo"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Lazio Rom", "club_home_pretty_name"
] = "Lazio"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Lazio Rom", "club_away_pretty_name"
] = "Lazio"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Genua Cfc", "club_home_pretty_name"
] = "genoa"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Genua Cfc", "club_away_pretty_name"
] = "genoa"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Cagliari Calcio",
    "club_home_pretty_name",
] = "Cagliari"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Cagliari Calcio",
    "club_away_pretty_name",
] = "Cagliari"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Sampdoria Genua",
    "club_home_pretty_name",
] = "Sampdoria"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Sampdoria Genua",
    "club_away_pretty_name",
] = "Sampdoria"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Delfino Pescara 1936",
    "club_home_pretty_name",
] = "Pescara"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Delfino Pescara 1936",
    "club_away_pretty_name",
] = "Pescara"


df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "1 Fsv Mainz 05", "club_home_pretty_name"
] = "Mainz 05"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "1 Fsv Mainz 05", "club_away_pretty_name"
] = "Mainz 05"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Ingolstadt 04",
    "club_home_pretty_name",
] = "Ingolstadt"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Ingolstadt 04",
    "club_away_pretty_name",
] = "Ingolstadt"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Sc Paderborn 07",
    "club_home_pretty_name",
] = "Paderborn"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Sc Paderborn 07",
    "club_away_pretty_name",
] = "Paderborn"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Sv Darmstadt 98",
    "club_home_pretty_name",
] = "Darmstadt"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Sv Darmstadt 98",
    "club_away_pretty_name",
] = "Darmstadt"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Bayern Munchen",
    "club_home_pretty_name",
] = "Bayern Munich"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Bayern Munchen",
    "club_away_pretty_name",
] = "Bayern Munich"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "1 Fc Koln", "club_home_pretty_name"
] = "FC Cologne"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "1 Fc Koln", "club_away_pretty_name"
] = "FC Cologne"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Hertha Bsc", "club_home_pretty_name"
] = "Hertha Berlin"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Hertha Bsc", "club_away_pretty_name"
] = "Hertha Berlin"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Tsg 1899 Hoffenheim",
    "club_home_pretty_name",
] = "Hoffenheim"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Tsg 1899 Hoffenheim",
    "club_away_pretty_name",
] = "Hoffenheim"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Hertha Bsc", "club_home_pretty_name"
] = "Hertha Berlin"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Hertha Bsc", "club_away_pretty_name"
] = "Hertha Berlin"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Hertha Bsc", "club_home_pretty_name"
] = "Hertha Berlin"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Hertha Bsc", "club_away_pretty_name"
] = "Hertha Berlin"


df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Palermo Fc", "club_home_pretty_name"
] = "Palermo"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Palermo Fc", "club_away_pretty_name"
] = "Palermo"

#-----------------------------------------------------------------------------------------------

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Girondins Bordeaux", "club_home_pretty_name"
] = "Bordeaux"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Girondins Bordeaux", "club_away_pretty_name"
] = "Bordeaux"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Stade Rennes", "club_home_pretty_name"
] = "Rennes"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Stade Rennes", "club_away_pretty_name"
] = "Rennes"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Fc Lorient", "club_home_pretty_name"
] = "Lorient"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Fc Lorient", "club_away_pretty_name"
] = "Lorient"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Losc Lille", "club_home_pretty_name"
] = "Lille"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Losc Lille", "club_away_pretty_name"
] = "Lille"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ogc Nizza", "club_home_pretty_name"
] = "Nizza"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ogc Nizza", "club_away_pretty_name"
] = "Nizza"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Thonon Evian Grand Geneve Fc", "club_home_pretty_name"
] = "Evian Thonon Gaillard"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Thonon Evian Grand Geneve Fc", "club_away_pretty_name"
] = "Evian Thonon Gaillard"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Stade Reims", "club_home_pretty_name"
] = "Reims"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Stade Reims", "club_away_pretty_name"
] = "Reims"

df2_games_top5.loc[
    df2_games_top5["club_home_pretty_name"] == "Ea Guingamp", "club_home_pretty_name"
] = "Guingamp"
df2_games_top5.loc[
    df2_games_top5["club_away_pretty_name"] == "Ea Guingamp", "club_away_pretty_name"
] = "Guingamp"

df2_games_top5["match_name"] = (
    df2_games_top5["club_home_pretty_name"]
    + " vs "
    + df2_games_top5["club_away_pretty_name"]
)




In [17]:
#----------------------------------------------------------
# despues de crear las columnas de los nombres de los match
# comparo los match de df_game con df2_games_top5
#---------------------------------------------------------

def match_games(x, y, score):

    liga = x["liga_new_id"]
    date = x["date"]
   

    y = y.loc[
        (y["leagueID"] == liga)
        & (y["date"] == date)
    
    ]

    match = process.extractOne(
        x["match_name"], y["match_name"], score_cutoff=score, scorer=fuzz.WRatio
    )

    if match:
        return match[2]
    else:
        return 12680


# ---------------------------------------------------------------------------------------------------------

df2_games_top5[["match_name_df", "homeTeamID", "awayTeamID",'home_name']] = df_f_game.loc[
    df2_games_top5.apply(lambda x: match_games(x, df_f_game, 85), axis=1).values,
    ["match_name", "homeTeamID", "awayTeamID",'home_name'],
].values
# ---------------------------------------------------------------------------------------------------------


In [18]:
#----------------------------------------------------------

# un nuevo dataframe con los id de clubes de df_game 
# con los id de clubes de df2_clubs, primero junte los 
# equipos en home con las columnas de id de los dos dataframes



club_agrupados_home = df2_games_top5[
    ["home_club_id", "club_home_pretty_name", "homeTeamID"]
]

gb_club_agrupados_home = club_agrupados_home.groupby(["home_club_id", "homeTeamID"])[
    "club_home_pretty_name"
].size()

gb_club_agrupados_home = (
    gb_club_agrupados_home.reset_index(level="home_club_id")
    .groupby("home_club_id")["club_home_pretty_name"]
    .idxmax()
    .reset_index(name="homeTeamID")
)
#---------------------------------------------------------

df2_clubs_top5_only_names = df2_clubs_top5[["club_id", "pretty_name"]]


gb_club_agrupados_home = pd.merge(
    gb_club_agrupados_home,
    df2_clubs_top5_only_names,
    left_on="home_club_id",
    right_on="club_id",
)


gb_club_agrupados_home = pd.merge(
    gb_club_agrupados_home, df_teams, left_on="homeTeamID", right_on="teamID"
)

# cree un dataset donde estan los nombres de los clubes con su id de los dos datasets df_teams y df2_clubes
gb_club_agrupados_home = gb_club_agrupados_home.drop(
    columns=["home_club_id", "homeTeamID", "pretty_name", "name"]
)



In [19]:
df_teams=pd.merge(df_teams,gb_club_agrupados_home)
df_teams=pd.merge(df_teams,df2_clubs_top5,on='club_id')


In [20]:

#cree un dataset separado de df_games y df2_games_top5 donde solo tengo las columnas de id y algunos datos

df2_games_top5_game_id=df2_games_top5[['game_id','date','home_club_id','away_club_id']]
df_f_game_game_id=df_f_game[['gameID','date','homeTeamID','awayTeamID']]
df_f_game_game_id.drop(index=12680,inplace=True)
#---------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------
#df2_games_top5 lo uno con el datasets de clubes donde ahora tiene los id del teams del datasets df_teams para poder hacer el merge asof
df2_games_top5_game_id=pd.merge(df2_games_top5_game_id,gb_club_agrupados_home,left_on='home_club_id',right_on='club_id')
df2_games_top5_game_id=pd.merge(df2_games_top5_game_id,gb_club_agrupados_home,left_on='away_club_id',right_on='club_id',suffixes=('_home','_away'))
df2_games_top5_game_id.drop(columns=['club_id_home','club_id_away'],inplace=True)
df2_games_top5_game_id.rename(columns={'teamID_home':"homeTeamID",'teamID_away':'awayTeamID'},inplace=True)


#--------------------------------------------
#preparo el datasets df_f_game
df_f_game_game_id['date']=pd.to_datetime(df_f_game_game_id.date)
df_f_game_game_id=df_f_game_game_id.sort_values(by='date')
df2_games_top5_game_id=df2_games_top5_game_id.sort_values(by='date')
df_f_game_game_id['homeTeamID']=df_f_game_game_id.homeTeamID.astype('int64')
df_f_game_game_id['awayTeamID']=df_f_game_game_id.awayTeamID.astype('int64')







In [21]:
df2_games_top5_game_id

,game_id,date,home_club_id,away_club_id,homeTeamID,awayTeamID
9098,2482105,2014-08-08,1421,583,177,161
8255,2482108,2014-08-09,855,618,169,175
9212,2482107,2014-08-09,14171,1162,209,165
7882,2482110,2014-08-09,969,40,166,176
9048,2482111,2014-08-09,995,826,168,210
...,...,...,...,...,...,...
11787,3433562,2021-05-23,3522,12,260,95
7881,3394999,2021-05-23,1421,40,177,176
2216,3451482,2021-05-23,368,1108,138,158
10372,3433564,2021-05-23,6195,276,105,94


In [22]:
df_f_game_game_id

,gameID,date,homeTeamID,awayTeamID
11221,6185.0,2014-08-08,177,161
11225,6189.0,2014-08-09,160,180
11228,6192.0,2014-08-09,170,174
11227,6191.0,2014-08-09,168,210
11226,6190.0,2014-08-09,166,176
...,...,...,...,...
2651,14806.0,2021-05-23,71,80
2650,14805.0,2021-05-23,83,220
5319,15805.0,2021-05-23,105,94
10119,15155.0,2021-05-23,138,158


In [23]:
#--------------------------------------------
#ahora uno el con la fecha mas cercana los df2_games y df_f_game y segun las columnas de hometeam y away
df2_games_top5_game_id=pd.merge_asof(df2_games_top5_game_id,df_f_game_game_id,on='date', by=['homeTeamID','awayTeamID'], direction='nearest',tolerance=pd.Timedelta(days=3))


df_f_game_game_id_date=df2_games_top5_game_id[['gameID','game_id','date']]
df_appearances=pd.merge(df_appearances,df_f_game_game_id_date)

df_appearances=pd.merge(df_appearances,df_players)


In [24]:
m = df2_appearances.select_dtypes(np.number)
df2_appearances[m.columns]= m.round().astype('Int64')

m = df_appearances.select_dtypes(np.number)
df_appearances[m.columns]= m.round().astype('Int64')


In [25]:
df2_appearances.drop(columns='player_pretty_name',inplace=True)
df2_col=df2_players[['player_id','pretty_name']]


In [26]:
df2_appearances=pd.merge(df2_appearances,df2_col)

#-----------------------------------------------
#cree una fila de nan para usarla en el match
nan_values = pd.DataFrame(np.nan, index=[0], columns=df2_appearances.columns)
df2_appearances = df2_appearances.append(nan_values, ignore_index=True)

In [27]:
"""   yellowCard = x["yellowCard"]
    redCard = x["redCard"]
    goals = x["goals"]
    assists = x["assists"] 
 & (y["yellow_cards"] == yellowCard)
        & (y["red_cards"] == redCard)
        & (y["goals"] == goals)
        & (y["assists"] == assists """

def match_games(x, y, score):

    game = x["game_id"]


    y = y.loc[y["game_id"] == game]

    match = process.extractOne(
        x["name"], y["pretty_name"], score_cutoff=score, scorer=fuzz.WRatio
    )

    if match:
        return match[2]
    else:
        return 355327


# ---------------------------------------------------------------------------------------------------------

df_appearances[['playerID_df2', "player_club_id", "appearance_id",'pretty_name']] = df2_appearances.loc[
    df_appearances.apply(lambda x: match_games(x, df2_appearances, 85), axis=1).values,
    ['player_id', "player_club_id", "appearance_id",'pretty_name'],
].values
# ---------------------------------------------------------------------------------------------------------


In [28]:
df_appearances = df_appearances[df_appearances['playerID_df2'].notna()]

In [29]:
club_agrupados_players = df_appearances[
    ["playerID", "player_club_id", "playerID_df2"]
]

gb_club_agrupados_players = club_agrupados_players.groupby(["playerID", "playerID_df2"])[
    "player_club_id"
].size()

gb_club_agrupados_players = (
    gb_club_agrupados_players.reset_index(level="playerID")
    .groupby("playerID")["player_club_id"]
    .idxmax()
    .reset_index(name="playerID_df2")
)
df2_players=df2_players[
    df2_players["player_id"].isin(df2_appearances["player_id"])
]


df_players=pd.merge(df_players,gb_club_agrupados_players,on='playerID',how='left')
gb_club_agrupados_players=pd.merge(gb_club_agrupados_players,df_players)

gb_club_agrupados_players=pd.merge(gb_club_agrupados_players,df2_players,left_on='playerID_df2',right_on='player_id')


In [30]:

df_teams['total_market_value']=df_teams['total_market_value'].fillna(df_teams['total_market_value'].median())


In [31]:


gb_club_agrupados_players.rename(columns={'name_x':'name'},inplace=True)

gb_club_agrupados_players.drop(columns=['name_y'],inplace=True)


In [32]:
df_f_game.to_csv('df_game_new.csv',index=False)
df_appearances.to_csv('df_appearances_new.csv',index=False)
df_teams.to_csv('df_teams_new.csv',index=False)
gb_club_agrupados_players.to_csv('df_players_new.csv',index=False)